# Autogen Workflow

Made up of:

```text
Workflow (1) -> (*) Agents
    Agents (1) -> (*) skills
```

References:

- [Autogenstudio](https://microsoft.github.io/autogen/docs/autogen-studio/getting-started)

## Install the required packages

In [1]:
%pip install -q python-dotenv==1.0.1 openai==1.35.9 gradio==4.39.0 pyautogen==0.2.32


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the required packages and create the Azure OpenAI client

In [ ]:
import os
import time
from datetime import datetime
from typing import Dict, Tuple, Union, Optional
from pathlib import Path
import autogen
from autogen import ConversableAgent, ChatResult
from autogen.coding import LocalCommandLineCodeExecutor
from dotenv import load_dotenv

## Loand the environment variables and prepare the Autogen configuration

In [ ]:
load_dotenv("../../.env")

model = os.getenv("GPT_MODEL")
endpoint=os.getenv("ENDPOINT")
api_key=os.getenv("API_KEY")
api_version=os.getenv("API_VERSION")

config_list = [
    {
        "base_url": endpoint,
        "api_key": api_key,
        "model": model,
        "api_type": "azure",
        "api_version": api_version
    }
]

llm_config = {
    "model": model,
    "temperature": 0,
    "config_list": config_list,    
    "cache_seed": None, # Enable caching
}

## Supporting classes and methods

The following supporting classes come from Autogen Studio: a tool "to help you rapidly prototype multi-agent solutions for your tasks, we are introducing AutoGen Studio, an interface powered by AutoGen."

1. `ExtendedConversableAgent`: Extends the ConversableAgent class with the ability to tap into the conversation between agents as it is happeing.
2. `process_message`: the function to use to tap into the messages.
3. `chat_history`: the history of the conversation being listen to.

Refences:

- [autogenstudio](https://microsoft.github.io/autogen/blog/2023/12/01/AutoGenStudio/)

In [ ]:
chat_history = []

def process_message(
    sender: autogen.Agent,
    receiver: autogen.Agent,
    message: Dict,
    request_reply: bool = False,
    silent: bool = False,
    sender_type: str = "agent",
) -> None:
    """
    Processes the message and adds it to the agent history.

    Args:

        sender: The sender of the message.
        receiver: The receiver of the message.
        message: The message content.
        request_reply: If set to True, the message will be added to agent history.
        silent: determining verbosity.
        sender_type: The type of the sender of the message.
    """

    message = message if isinstance(message, dict) else {
        "content": message, "role": "user"}
    message_payload = {
        "recipient": receiver.name,
        "sender": sender.name,
        "message": message,
        "timestamp": datetime.now().isoformat(),
        "sender_type": sender_type,
        "message_type": "agent_message",
    }
    if message["content"]:
        #print(message_payload)
        chat_history.append(message_payload)


class ExtendedConversableAgent(ConversableAgent):
    def __init__(self, message_processor=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.message_processor = message_processor
        self.history = []

    def receive(
        self,
        message: Union[Dict, str],
        sender: autogen.Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        if self.message_processor:
            self.message_processor(sender, self, message,
                                   request_reply, silent, sender_type="agent")
        # print(f"Sender: {sender.name}")
        # print(f"Message: {message}")
        super().receive(message, sender, request_reply, silent)

## Configure the local Autogen code executor

In [ ]:
# Configure the code executor
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

## Config the Autogen agents

In [ ]:
# Create the agent without an LLM but with code execution capabilities.
user = ExtendedConversableAgent(name="user",
                                max_consecutive_auto_reply=5,
                                code_execution_config={"executor": executor},
                                human_input_mode="NEVER",
                                is_termination_msg=lambda msg: "TERMINATE" in msg["content"].lower() or msg["content"]=="",
                                message_processor=process_message,
                                )

# Create the agent that uses the LLM.
assistant = ExtendedConversableAgent(
    name="assistant",
    system_message="You are a general AI assistant that can answer questions and generate code.",
    llm_config=llm_config,    
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"].lower() or msg["content"]=="" or "exitcode: 0" in msg["content"],
    message_processor=process_message,
)

## Process a chat conversation

In [ ]:
def chat(message: str,clear=False,silent=True,max_turns=-1)->Tuple[float,str]:
    if clear:
        chat_history.clear()
    start_time = time.time()
    # The conversation history is being recorded by the process_message function in the chat_history array
    result : ChatResult = None
    if max_turns>0:
        result = user.initiate_chat(assistant,message=message,clear_history=clear,silent=silent,max_turns=max_turns)
    else:
        result = user.initiate_chat(assistant,message=message,clear_history=clear,silent=silent)
    end_time = time.time()
    duration = end_time - start_time
    return (duration,result.summary)
    

## Print the conversation history

In [ ]:
def print_history()->None:
    for message in chat_history:
        print(f"{message['sender']} : {message['message']['content']}")

## Have a conversation between the user and the assistant

In [ ]:
chat("List is one good restaurant in Miami.",clear=True)
chat("What is another one?")
chat("Write a python app to find 1001st prime number?")



## Print the conversation history

In [ ]:
print_history()